In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 4
EPOCHS = 20

TRAIN_DIR = "/Users/hrithickkanagaraj/Documents/Programming /ML/Damage_classification/CNN/data_set/train"
VAL_DIR   = "/Users/hrithickkanagaraj/Documents/Programming /ML/Damage_classification/CNN/data_set/val"
TEST_DIR  = "/Users/hrithickkanagaraj/Documents/Programming /ML/Damage_classification/CNN/data_set/test"

FINAL_MODEL_PATH = "cnnn_damage_model.h5"


In [4]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_data = val_test_gen.flow_from_directory(
    TEST_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("Class mapping:", train_data.class_indices)


Found 96 images belonging to 4 classes.
Found 25 images belonging to 4 classes.
Found 30 images belonging to 4 classes.
Class mapping: {'light_broken': 0, 'moderately_broken': 1, 'no_broken': 2, 'severe_broken': 3}


In [5]:
input_tensor = Input(shape=(224, 224, 3))

base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_tensor=input_tensor
)

base_model.trainable = False  # Phase 1

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=input_tensor, outputs=outputs)


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)




In [8]:
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks
)


/opt/miniconda3/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 665ms/step - accuracy: 0.2292 - loss: 2.3676 - val_accuracy: 0.4800 - val_loss: 1.1496
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 493ms/step - accuracy: 0.3438 - loss: 1.9794 - val_accuracy: 0.5600 - val_loss: 0.9046
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 487ms/step - accuracy: 0.4688 - loss: 1.4132 - val_accuracy: 0.6400 - val_loss: 0.7476
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 488ms/step - accuracy: 0.4896 - loss: 1.2792 - val_accuracy: 0.8000 - val_loss: 0.6043
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 490ms/step - accuracy: 0.5729 - loss: 1.0657 - val_accuracy: 0.7200 - val_loss: 0.5262
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 492ms/step - accuracy: 0.5625 - loss: 1.2459 - val_accuracy: 0.8000 - val_loss: 0.4830
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 499ms/step - accuracy: 0.6562 - loss: 0.8968 - val_accuracy: 0.8400 - val_loss: 0.4384
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 495ms/step - accuracy: 0.6771 - loss: 0.9057 - val_accuracy: 0.8000 - val_loss:

In [9]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.6354 - loss: 0.9344 - val_accuracy: 0.9600 - val_loss: 0.2459
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6250 - loss: 1.0672 - val_accuracy: 0.9200 - val_loss: 0.2324
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6562 - loss: 0.8595 - val_accuracy: 0.9200 - val_loss: 0.2260
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6667 - loss: 0.7519 - val_accuracy: 0.9200 - val_loss: 0.2202
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7604 - loss: 0.5783 - val_accuracy: 0.8800 - val_loss: 0.2165
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7396 - loss: 0.6761 - val_accuracy: 0.8800 - val_loss: 0.2151
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7604 - loss: 0.6427 - val_accuracy: 0.8800 - val_loss: 0.2133
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8125 - loss: 0.5320 - val_accuracy: 0.9200 - val_loss: 0.2095
Epoch 9/

In [10]:
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.8333 - loss: 0.5791
Test Accuracy: 0.8333


In [11]:
model.save(FINAL_MODEL_PATH)
print("✅ Model saved as cnn_damage_model.h5")


✅ Model saved as cnn_damage_model.h5


In [12]:
from tensorflow.keras.models import load_model

m = load_model("cnn_damage_model.h5", compile=False)
print("✅ Model reload test passed")


✅ Model reload test passed


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

print(tf.__version__)  # should be 2.15.0

model = load_model("final_resnet50_damage_model.h5", compile=False)

model.save("cnnn_damage_model.h5")   # overwrite safely
print("✅ Model re-saved successfully")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    a

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    a

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/tf_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    a

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import